# PyTorch Test for ECG Data

In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
# Import Numpy
import numpy as np
# Plot Import
import matplotlib.pyplot as plt
# For timer
import time
# Import OS
import os
# import local libs
import ecg_tools_lite as et

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [13]:
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device, time.localtime().tm_mon, "/", time.localtime().tm_mday, "/", time.localtime().tm_year)

Device: cuda 11 / 13 / 2020


In [14]:
ecg_clean = et.load_ecg_file('all_chunks_fs1024_e24.npy')
ecg_noisy = et.load_ecg_file('all_chunks_fs1024_e06.npy')

trainloader = DataLoader( ecg_noisy, shuffle=True, num_workers=2 )
# testloader = DataLoader()

In [4]:
ecg_clean.shape

(15840, 360, 1)

In [5]:
num_epochs = 1
batch_size = 128
learning_rate = 1e-3

def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)

In [7]:
batch_size_data = 1
length_data = 1
dimension_data =  1024

In [6]:
in_chan = 1024
out_chan = 1
kern_size = 2

class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(in_channels=in_chan, out_channels=out_chan, kernel_size=kern_size),
            nn.ReLU(True),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose1d(in_channels=in_chan, out_channels=out_chan, kernel_size=kern_size),
            nn.ReLU(True),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
                model.parameters()
            ) # lr=learning_rate, weight_decay=1e-5)

TypeError: __init__() missing 1 required positional argument: 'kernel_size'

In [25]:
ecg_rand = torch.rand(1,360,1)
trainloader = DataLoader( ecg_rand, shuffle=True)

In [26]:
ecg_rand.size()

torch.Size([1, 360, 1])

In [27]:
# Added elapsed timer
elapsed_start = time.time()
# Device cuda
device = torch.device("cuda")
# Start training
for epoch in range(num_epochs):
    # running Loss
    running_loss = 0.0
    # Epoch timer start
    epoch_start = time.time()
    print( "Starting epoch: ", epoch)
    for i, data in enumerate(ecg_rand, 0):        
        # zero the gradients
        optimizer.zero_grad()
        # NO idea
        # data_temp = data.view(360)
        # outputs.to(device)
        outputs = model( data )
        loss = criterion(data, ecg_clean)
        loss.backward()
        optimizer.step()
        # Running Loss
        running_loss += loss.item()
    # ===== LOGS =====
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data.item(), MSE_loss.data.item()
          )
          )
    epoch_end = time.time()
    print("Epoch run time: %.2f" %(epoch_end-epoch_start))
# ===== Total training elapsed time =====
elapsed_end = time.time()
elapsed_total = elapsed_end-elapsed_start
print("Total elapsed time: %.2f, (in minutes %3d)" %(elapsed_total, (elapsed_total/60)))

print("Finished Training")


Starting epoch:  0


RuntimeError: Expected 3-dimensional input for 3-dimensional weight [1, 360, 2], but got 2-dimensional input of size [360, 1] instead